In [28]:
import pandas as pd
import numpy as np

from tensorflow.keras.callbacks import EarlyStopping

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn import datasets, linear_model
import matplotlib.pyplot as plt
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


import tensorflow 
import seaborn as sns

from tensorflow import keras
from keras.layers import Dense
from sklearn.metrics import classification_report, confusion_matrix

import cv2

import numpy as np


from sklearn.linear_model import LinearRegression

import joblib

In [3]:
# Definir la ruta del dataset
DATASET_PATH = r"C:\Users\javid\OneDrive\Escritorio\Javidev\DS_Bootcamp\DS_Online_Octubre24\05_Deep_Learning\Sprint_17\Team_Challenge\asl_dataset"

In [4]:
# Obtener las clases disponibles (carpetas dentro del dataset)
clases = sorted(os.listdir(DATASET_PATH))  # ['0', '1', ..., '9', 'A', 'B', ..., 'Z']
num_clases = len(clases)

X = []
y = []

# Cargar imágenes y convertirlas a arrays
for label, clase in enumerate(clases):
    folder_path = os.path.join(DATASET_PATH, clase)
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Convertimos a escala de grises
        img = cv2.resize(img, (64, 64))  # Redimensionamos a 64x64 píxeles
        X.append(img.flatten())  # Convertimos la imagen en un vector
        y.append(label)  # Guardamos la etiqueta de la clase

# Convertir a arrays de numpy
X = np.array(X)
y = np.array(y)

# Mezclar los datos aleatoriamente
indices = np.arange(len(X))
np.random.shuffle(indices)

# Aplicar el desorden a X e y
X = X[indices]
y = y[indices]

# Dividir en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Guardar los datos para uso futuro
np.save("asl_features_train.npy", X_train)
np.save("asl_labels_train.npy", y_train)
np.save("asl_features_test.npy", X_test)
np.save("asl_labels_test.npy", y_test)

print(f"Datos cargados y mezclados: {X.shape[0]} imágenes, {num_clases} clases")
print(f"Train: {X_train.shape[0]} imágenes | Test: {X_test.shape[0]} imágenes")


Datos cargados y mezclados: 2515 imágenes, 36 clases
Train: 2012 imágenes | Test: 503 imágenes


In [5]:
X_train.shape

(2012, 4096)

In [6]:
X_test.shape

(503, 4096)

In [7]:
lr = LinearRegression()

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=33)

In [9]:
lasso = linear_model.Lasso()


pipe = Pipeline([('Linear', LinearRegression())])

pipe2 = Pipeline([('Standard Scaler', StandardScaler())])

In [10]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

def ejecutar(pipe, X_train, X_test, y_train, y_test):
    pipe.fit(X_train, y_train) 
    pred = pipe.predict(X_test)  
    
    # Calcula y muestra el error cuadrático medio
    mse = mean_squared_error(y_test, pred)
    print(f"MSE = {mse}")

    # Validación cruzada usando el propio pipeline
    cv_scores = cross_val_score(pipe, X_train, y_train, cv=5, scoring="neg_mean_squared_error")
    print(f"Cross-validation MSE: {-cv_scores.mean()}") 


In [11]:
def ejecutar2(pipe, X_train, X_test, y_train, y_test):
    pipe.fit(X_train, y_train)
    
    # Mostrar solo una parte de X_train para evitar impresión masiva
    if hasattr(X_train, "head"):  # Si es un DataFrame de pandas
        print(X_train.head())
    else:  # Si es un array de NumPy
        print(X_train[:5])


In [12]:
ejecutar(pipe, X_train, X_test, y_train, y_test)

MSE = 67.4692643374758
Cross-validation MSE: 53.236522723609724


In [13]:
ejecutar2(pipe2, X_train, X_test, y_train, y_test)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [14]:
pipe2.fit(X_train, y_train)

Pipeline(steps=[('Standard Scaler', StandardScaler())])

In [15]:
pipe

Pipeline(steps=[('Linear', LinearRegression())])

In [16]:
model = keras.Sequential([ 
    keras.layers.Dense(128, activation='relu'),  
    keras.layers.Dense(64, activation='relu'),   
    keras.layers.Dense(36, activation='softmax') # Capa de salida con 10 neuronas (10 clases)
])

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify=y)

In [18]:
pipeline = Pipeline([('model', model)])

In [19]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],
             )


In [20]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


model.fit(X_train, y_train, epochs=36, validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/36
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.1514 - loss: 88.3545 - val_accuracy: 0.4235 - val_loss: 6.4237
Epoch 2/36
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.4844 - loss: 4.8313 - val_accuracy: 0.6421 - val_loss: 2.6583
Epoch 3/36
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6391 - loss: 2.2127 - val_accuracy: 0.5686 - val_loss: 3.0607
Epoch 4/36
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6973 - loss: 1.5863 - val_accuracy: 0.6262 - val_loss: 2.9526
Epoch 5/36
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6597 - loss: 2.3098 - val_accuracy: 0.7833 - val_loss: 1.2115
Epoch 6/36
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7960 - loss: 0.9544 - val_accuracy: 0.7753 - val_loss: 1.6191
Epoch 7/36
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8388 - loss: 0.6432 - val_accuracy: 0.7972 - val_loss: 0.9871
Epoch 8/36
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8642 - loss: 0.5412 - val_accuracy: 0.7614 - val_loss

In [21]:

from sklearn.cluster import KMeans

# Ejemplo de KMeans con 3 clusters
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(X)
labels = kmeans.predict(X)


In [22]:

from sklearn.preprocessing import OneHotEncoder

# OneHotEncoder con manejo de categorías nuevas en test
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
X_train_encoded = encoder.fit_transform(X_train)
X_test_encoded = encoder.transform(X_test)


In [23]:

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Definir modelo y parámetros
model = RandomForestClassifier(random_state=42)
param_grid = {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20]}

# GridSearchCV
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Mejor modelo
best_model = grid_search.best_estimator_
print("Mejores parámetros:", grid_search.best_params_)


Mejores parámetros: {'max_depth': None, 'n_estimators': 200}


In [24]:

from imblearn.over_sampling import SMOTE

# Aplicar SMOTE para balancear clases en el conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)


In [27]:
save_dir = ("../models")

In [29]:
model_path = os.path.join(save_dir, "random_forest_model.pkl")
joblib.dump(model, model_path)
print(f"Modelo guardado en: {model_path}")

Modelo guardado en: ../models\random_forest_model.pkl


In [ ]:
# loaded_model = joblib.load(model_path)
# print("Modelo cargado correctamente")